<a href="https://colab.research.google.com/github/refelxgod/Movie_recommender_sys/blob/main/Final_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2024-05-05 19:24:39--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-05-05 19:24:39--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-05-05 19:24:39--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
!ls
!pwd

clean_d_tweets.csv	glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
clean_non_d_tweets.csv	glove.6B.200d.txt  glove.6B.50d.txt   sample_data
/content


LSTM used

Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

Load the datasets

In [ ]:
depressed_tweets = pd.read_csv('/content/clean_d_tweets.csv')
non_depressed_tweets = pd.read_csv('/content/clean_non_d_tweets.csv')


Combine the datasets

In [ ]:
df = pd.concat([depressed_tweets, non_depressed_tweets])


Create a column for the labels

In [ ]:
df['label'] = np.where(df['tweet'].isin(depressed_tweets['tweet']), 1, 0)

Preprocess the tweets

In [ ]:
def clean_tweets(text):
    if isinstance(text, str):  # Check if the value is a string
        text = re.sub(r'[^A-Za-z0-9\\s]', '', text.lower())
    else:
        text = str(text)  # Convert non-string values to strings
    return text

df['tweet'] = df['tweet'].apply(clean_tweets)

df['tweet'] = df['tweet'].apply(clean_tweets)

Split the data into training and testing sets

In [ ]:
train_text, val_text, train_labels, val_labels = train_test_split(df['tweet'].to_numpy(), df['label'].to_numpy(), test_size=0.2, random_state=42)


Create a tokenizer to split the text into words

In [ ]:
tokenizer = Tokenizer(num_words=3000)
tokenizer.fit_on_texts(train_text)

train_sequences = tokenizer.texts_to_sequences(train_text)
val_sequences = tokenizer.texts_to_sequences(val_text)

Pad the sequences to have the same length

In [ ]:
max_length = 200
padded_train = pad_sequences(train_sequences, maxlen=max_length)
padded_val = pad_sequences(val_sequences, maxlen=max_length)

One-hot encode the labels

In [ ]:
num_classes = 2
train_labels_onehot = to_categorical(train_labels, num_classes)
val_labels_onehot = to_categorical(val_labels, num_classes)

Load the pre-trained GloVe word embeddings

In [ ]:
glove_file = '/content/glove.6B.300d.txt'
embeddings_index = {}
with open(glove_file, 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

Create the embedding matrix

In [ ]:
embedding_matrix = np.zeros((3000, 300))
for word, i in tokenizer.word_index.items():
    if i < 3000:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

Define the LSTM model

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=3000, output_dim=300, input_length=max_length, weights=[embedding_matrix], trainable=False))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

Compile the model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Train the model

In [ ]:
batch_size = 500
epochs = 25
history = model.fit(padded_train, train_labels_onehot, batch_size=batch_size, epochs=epochs, validation_data=(padded_val, val_labels_onehot))


Epoch 1/25
13/13 [==============================] - 11s 625ms/step - loss: 0.6779 - accuracy: 0.6069 - val_loss: 0.6747 - val_accuracy: 0.5940
Epoch 2/25
13/13 [==============================] - 14s 1s/step - loss: 0.6708 - accuracy: 0.6087 - val_loss: 0.6738 - val_accuracy: 0.5940
Epoch 3/25
13/13 [==============================] - 7s 560ms/step - loss: 0.6681 - accuracy: 0.6088 - val_loss: 0.6744 - val_accuracy: 0.5940
Epoch 4/25
13/13 [==============================] - 14s 1s/step - loss: 0.6671 - accuracy: 0.6088 - val_loss: 0.6738 - val_accuracy: 0.5940
Epoch 5/25
13/13 [==============================] - 8s 552ms/step - loss: 0.6672 - accuracy: 0.6090 - val_loss: 0.6740 - val_accuracy: 0.5940
Epoch 6/25
13/13 [==============================] - 11s 901ms/step - loss: 0.6669 - accuracy: 0.6090 - val_loss: 0.6743 - val_accuracy: 0.5940
Epoch 7/25
13/13 [==============================] - 11s 823ms/step - loss: 0.6667 - accuracy: 0.6090 - val_loss: 0.6739 - val_accuracy: 0.5940
Epoch 8

Evaluate the model

In [ ]:
loss, accuracy = model.evaluate(padded_val, val_labels_onehot)
print(f'Test accuracy: {accuracy:.3f}')

49/49 [==============================] - 5s 94ms/step - loss: 0.6738 - accuracy: 0.5940
Test accuracy: 0.594


Bidirectional LSTM

In [ ]:
from tensorflow.keras.layers import Bidirectional

# Define the Bidirectional LSTM model
model = Sequential()
model.add(Embedding(input_dim=3000, output_dim=300, input_length=max_length, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(padded_train, train_labels_onehot, batch_size=batch_size, epochs=epochs, validation_data=(padded_val, val_labels_onehot))

# Evaluate the model
loss, accuracy = model.evaluate(padded_val, val_labels_onehot)
print(f'Test accuracy: {accuracy:.3f}')

Epoch 1/25
13/13 [==============================] - 29s 2s/step - loss: 0.6779 - accuracy: 0.6060 - val_loss: 0.6744 - val_accuracy: 0.5940
Epoch 2/25
13/13 [==============================] - 18s 1s/step - loss: 0.6686 - accuracy: 0.6074 - val_loss: 0.6743 - val_accuracy: 0.5933
Epoch 3/25
13/13 [==============================] - 26s 2s/step - loss: 0.6682 - accuracy: 0.6084 - val_loss: 0.6748 - val_accuracy: 0.5933
Epoch 4/25
13/13 [==============================] - 22s 2s/step - loss: 0.6678 - accuracy: 0.6085 - val_loss: 0.6742 - val_accuracy: 0.5933
Epoch 5/25
13/13 [==============================] - 21s 2s/step - loss: 0.6674 - accuracy: 0.6088 - val_loss: 0.6745 - val_accuracy: 0.5933
Epoch 6/25
13/13 [==============================] - 21s 2s/step - loss: 0.6681 - accuracy: 0.6088 - val_loss: 0.6739 - val_accuracy: 0.5940
Epoch 7/25
13/13 [==============================] - 19s 2s/step - loss: 0.6669 - accuracy: 0.6090 - val_loss: 0.6743 - val_accuracy: 0.5940
Epoch 8/25
13/13 [==

GRU

In [ ]:
from tensorflow.keras.layers import GRU

# Define the GRU model
model = Sequential()
model.add(Embedding(input_dim=3000, output_dim=300, input_length=max_length, weights=[embedding_matrix], trainable=False))
model.add(GRU(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(padded_train, train_labels_onehot, batch_size=batch_size, epochs=epochs, validation_data=(padded_val, val_labels_onehot))

# Evaluate the model
loss, accuracy = model.evaluate(padded_val, val_labels_onehot)
print(f'Test accuracy: {accuracy:.3f}')

Epoch 1/25
13/13 [==============================] - 10s 519ms/step - loss: 0.6770 - accuracy: 0.6072 - val_loss: 0.6777 - val_accuracy: 0.5940
Epoch 2/25
13/13 [==============================] - 12s 971ms/step - loss: 0.6678 - accuracy: 0.6085 - val_loss: 0.6739 - val_accuracy: 0.5933
Epoch 3/25
13/13 [==============================] - 8s 596ms/step - loss: 0.6670 - accuracy: 0.6088 - val_loss: 0.6746 - val_accuracy: 0.5940
Epoch 4/25
13/13 [==============================] - 7s 566ms/step - loss: 0.6669 - accuracy: 0.6088 - val_loss: 0.6741 - val_accuracy: 0.5940
Epoch 5/25
13/13 [==============================] - 13s 1s/step - loss: 0.6666 - accuracy: 0.6090 - val_loss: 0.6737 - val_accuracy: 0.5940
Epoch 6/25
13/13 [==============================] - 7s 521ms/step - loss: 0.6666 - accuracy: 0.6090 - val_loss: 0.6744 - val_accuracy: 0.5940
Epoch 7/25
13/13 [==============================] - 12s 930ms/step - loss: 0.6667 - accuracy: 0.6090 - val_loss: 0.6737 - val_accuracy: 0.5940
Epoch

Biderectional GRU

In [ ]:
from tensorflow.keras.layers import Bidirectional, GRU

# Define the Bidirectional GRU model
model = Sequential()
model.add(Embedding(input_dim=3000, output_dim=300, input_length=max_length, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(GRU(units=128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(padded_train, train_labels_onehot, batch_size=batch_size, epochs=epochs, validation_data=(padded_val, val_labels_onehot))

# Evaluate the model
loss, accuracy = model.evaluate(padded_val, val_labels_onehot)
print(f'Test accuracy: {accuracy:.3f}')

Epoch 1/25
13/13 [==============================] - 26s 2s/step - loss: 0.6746 - accuracy: 0.6071 - val_loss: 0.6772 - val_accuracy: 0.5940
Epoch 2/25
13/13 [==============================] - 20s 2s/step - loss: 0.6689 - accuracy: 0.6085 - val_loss: 0.6739 - val_accuracy: 0.5933
Epoch 3/25
13/13 [==============================] - 17s 1s/step - loss: 0.6673 - accuracy: 0.6087 - val_loss: 0.6756 - val_accuracy: 0.5940
Epoch 4/25
13/13 [==============================] - 23s 2s/step - loss: 0.6671 - accuracy: 0.6088 - val_loss: 0.6739 - val_accuracy: 0.5933
Epoch 5/25
13/13 [==============================] - 20s 2s/step - loss: 0.6671 - accuracy: 0.6088 - val_loss: 0.6741 - val_accuracy: 0.5940
Epoch 6/25
13/13 [==============================] - 21s 2s/step - loss: 0.6666 - accuracy: 0.6090 - val_loss: 0.6739 - val_accuracy: 0.5940
Epoch 7/25
13/13 [==============================] - 17s 1s/step - loss: 0.6666 - accuracy: 0.6090 - val_loss: 0.6738 - val_accuracy: 0.5940
Epoch 8/25
13/13 [==